In [3]:
import tensorflow as tf
import os
import pandas as pd

# Read the label DataFrame
label_df = pd.read_excel('C:\\Fetal_Brain\\FETAL_PLANES_DB_data.xlsx')
# Read the augmented DataFrame
augmented_df = pd.read_csv('C:\\Fetal_Brain\\AugmentedImages.csv')

def rotate_image(image, angle):
    """Rotate the image by a given angle."""
    rotated_image = tf.image.rot90(image, k=angle // 45)
    return rotated_image

def flip_image(image, flip_code):
    """Flip the image horizontally or vertically."""
    flipped_image = tf.image.flip_left_right(image) if flip_code == 1 else tf.image.flip_up_down(image)
    return flipped_image

# Directory containing images to be augmented
input_dir = "C:\\Fetal_Brain\\CroppedImages\\"
# Directory to save augmented images
save_dir = "C:\\Fetal_Brain\\AugmentedImages\\"

# Create the directory if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

# Augmentation parameters
rotation_angles = [0,45]  # Rotate by 45 degrees increments
flip_codes = [0, 1, 2]  # Flip horizontally, vertically, and both

# Iterate over each image in the directory
for image_file in os.listdir(input_dir):
    if image_file.endswith('.png'):
        # Load the image
        image_path = os.path.join(input_dir, image_file)
        image = tf.image.decode_image(tf.io.read_file(image_path), channels=1)
        # Get label for the image
        label = label_df['Train '][label_df['Image_name'] == os.path.basename(image_path).split('.')[0]].values[0]

        # Apply augmentation
        augmented_images = []
        for angle in rotation_angles:
            rotated_image = rotate_image(image, angle)  # Skip resizing here
            for flip_code in flip_codes:
                flipped_image = flip_image(rotated_image, flip_code)
                augmented_images.append(flipped_image)

        # Save augmented images
        for i, augmented_image in enumerate(augmented_images):
            # Convert image to uint8
            augmented_image = tf.image.convert_image_dtype(augmented_image, dtype=tf.uint8)
            # Generate filename
            filename = os.path.basename(image_path).split('.')[0] + f"_augmented_{i}.png"
            # Save image
            tf.io.write_file(os.path.join(save_dir, filename), tf.image.encode_png(augmented_image))
            # Update augmented DataFrame
            augmented_df = augmented_df.append({'Image_name': filename, 'Train': label}, ignore_index=True)

# Save the updated augmented DataFrame
augmented_df.to_csv('C:\\Fetal_Brain\\AugmentedImages.csv', index=False)


In [4]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the preprocessed images and corresponding labels
image_directory = 'C:\\Fetal_Brain\\AugmentedImages'
labels_file = 'C:\\Fetal_Brain\\AugmentedImages.csv'
labels_df = pd.read_csv(labels_file)

# Keep only the 'Image_name' and 'Train' columns
labels_df = labels_df[['Image_name', 'Train']]

# Prepare the data
images = []
labels = []
for filename in os.listdir(image_directory):
    image_path = os.path.join(image_directory, filename)
    if os.path.isfile(image_path):
        image_name = filename.split(".")[0] 
        label_row = labels_df.loc[labels_df['Image_name'] == image_name]
        if not label_row.empty:
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read image in grayscale
            resized_image = cv2.resize(image, (224, 224))
            images.append(resized_image.flatten())
            labels.append(label_row.iloc[0]['Train'])

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
# Train a Random Forest classifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train)
# Predictions
y_pred = rf_classifier.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)    


Accuracy: 0.7924151696606786


### CNN

In [5]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

# Load the preprocessed images and corresponding labels
image_directory = 'C:\\Fetal_Brain\\AugmentedImages'
labels_file = 'C:\\Fetal_Brain\\AugmentedImages.csv'
labels_df = pd.read_csv(labels_file)

# Prepare the data
images = []
labels = []
for filename in os.listdir(image_directory):
    image_path = os.path.join(image_directory, filename)
    if os.path.isfile(image_path):
        image_name = filename.split(".")[0] 
        label_row = labels_df.loc[labels_df['Image_name'] == image_name]
        if not label_row.empty:
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read image in grayscale
            resized_image = cv2.resize(image, (224, 224))
            images.append(resized_image)
            labels.append(label_row.iloc[0]['Train'])

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Normalize pixel values to be between 0 and 1
X_train_normalized = X_train / 255.0
X_test_normalized = X_test / 255.0

# Train the model
model.fit(X_train_normalized, y_train, epochs=10, batch_size=32, validation_data=(X_test_normalized, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test_normalized, y_test)
print('Test accuracy:', test_acc)


Epoch 1/10
63/63 [==============================] - 61s 946ms/step - loss: 0.6927 - accuracy: 0.5642 - val_loss: 0.6657 - val_accuracy: 0.5848
Epoch 2/10
63/63 [==============================] - 58s 913ms/step - loss: 0.5948 - accuracy: 0.6887 - val_loss: 0.9735 - val_accuracy: 0.5489
Epoch 3/10
63/63 [==============================] - 58s 916ms/step - loss: 0.4727 - accuracy: 0.7821 - val_loss: 0.6052 - val_accuracy: 0.6846
Epoch 4/10
63/63 [==============================] - 58s 913ms/step - loss: 0.2299 - accuracy: 0.9060 - val_loss: 0.8912 - val_accuracy: 0.7285
Epoch 5/10
63/63 [==============================] - 64s 1s/step - loss: 0.0785 - accuracy: 0.9725 - val_loss: 1.0203 - val_accuracy: 0.7764
Epoch 6/10
63/63 [==============================] - 68s 1s/step - loss: 0.0286 - accuracy: 0.9895 - val_loss: 1.0241 - val_accuracy: 0.7585
Epoch 7/10
63/63 [==============================] - 61s 972ms/step - loss: 0.0136 - accuracy: 0.9965 - val_loss: 1.3723 - val_accuracy: 0.7884
Epoch